In [1]:
import os
import math
import json
import gzip
import time
import calendar
from datetime import datetime
import numpy as np
import pandas as pd
from datetime import date
from multiprocessing import Pool
import warnings
warnings.filterwarnings('ignore')
SETS = './sets/' # папка с файлами с наборами
LONG = 0.0159 # долгота расстояние 1 км от точки на восток или на запад
LAT = 0.009 # широта расстояние 1 км от точки на север или на юг

In [2]:
def default_serializer(obj): # для правильного преобразования даты в ISO формат
    if isinstance(obj, (date)):
        return obj.isoformat()
    raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')
    

def load_dataset(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return pd.DataFrame(json.load(gzip_ref))
        
    
def save_dataset(filename):
    data = globals().get(filename)
    if data is not None:
        data = data.to_dict(orient='records')
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(data)} записей в {filename}.gz')
        

def distance(lon1, lat1, lon2, lat2):
    """
    Функция для вычисления расстояния между двумя точками на плоскости по их GPS координатам
    """
    R = 6371  # Радиус Земли в километрах
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) * np.sin(dlat / 2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) * np.sin(dlon / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c


def km_min(exams, ecology):
    counter = 0
    km_list = []
    for i, row in ecology.iterrows():
        if exams['datesSet'].intersection(row['datesSet']):
            a_list = exams['geoList']
            for a in range(0, len(a_list), 2):
                a_longlat = a_list[a:a+2]
                if len(a_longlat) >= 2: # если в списке больше двух точек
                    a_long = a_longlat[0]
                    a_lat = a_longlat[1]
                b_list = row['geoList']
                for b in range(0, len(b_list), 2):
                    b_longlat = b_list[b:b+2]
                    if len(b_longlat) >= 2:
                        b_long = b_longlat[0]
                        b_lat = b_longlat[1]
                    km = distance(a_long, a_lat, b_long, b_lat)
                    if km < 1:
                        counter += 1
                        km_list.append(km)
                        return 1-math.prod(km_list)
                        #print(f'расстояние {km} между [ {a_long}, {a_lat} ] и [ {b_long}, {b_lat} ]')
    return 0
    

def geo_list(geodata):
    if type(geodata) == str:
        try: 
            geodata = eval(geodata)
        except Exeption as e:
            print(geodata)
    geolist = []
    try:
        geolist = list(map(float, str(geodata['coordinates']).replace(',[]', '').replace('[', '').replace(']', '').split(',')))
    except Exception as e:
        print(e, geodata)
    return geolist[0:2]


def dates_set(start, end):
    exams_range = pd.date_range(start=start, end=end)
    return set(exams_range)


def sets_intersects(exams_set, start, end):
    """
    Для уменьшения объема датасета ecology, проводим проверку, входят ли даты событий в наборы дат экзаменов
    """
    date_range = pd.date_range(start=start, end=end)
    date_set = set(date_range)
    if date_set.intersection(exams_set):
        return True
    else:
        return False

In [3]:
pd.set_option('display.max_colwidth', None) # для отображения полного текста в ячейках
pd.set_option('display.max_columns', None) 

In [4]:
%%time
# Загружаем и обрабатываем облегченный вариант датасета экзаменов (не все параметры)
exams_lite = load_dataset('exams_lite')
exams_lite['datesSet'] = exams_lite.apply(lambda x: dates_set(x['start'], x['end']), axis=1)
exams_lite['geoList'] = exams_lite.apply(lambda x: geo_list(x['geoData']), axis=1)

CPU times: user 5.67 s, sys: 143 ms, total: 5.82 s
Wall time: 5.83 s


In [5]:
%%time
# Стройки
ecology_stroi = load_dataset('ecology_stroi')
ecology_stroi['datesSet'] = ecology_stroi.apply(lambda x: dates_set(x['start'], x['end']), axis=1)
ecology_stroi['geoList'] = ecology_stroi.apply(lambda x: geo_list(x['stroi']), axis=1)
exams_lite['stroi'] = exams_lite.apply(lambda x: km_min(x, ecology_stroi), axis=1)

CPU times: user 9min 16s, sys: 839 ms, total: 9min 17s
Wall time: 9min 19s


In [6]:
%%time
# Дороги
ecology_roads = load_dataset('ecology_roads')
ecology_roads['datesSet'] = ecology_roads.apply(lambda x: dates_set(x['start'], x['end']), axis=1)
ecology_roads['geoList'] = ecology_roads.apply(lambda x: geo_list(x['roads']), axis=1)
exams_lite['roads'] = exams_lite.apply(lambda x: km_min(x, ecology_roads), axis=1)

CPU times: user 20min 2s, sys: 257 ms, total: 20min 2s
Wall time: 20min 6s


In [8]:
exams_ecology = exams_lite[['global_id', 'start', 'end', 'stroi', 'roads', 'stupid']]
save_dataset('exams_ecology')

Сохранено 5542 записей в exams_ecology.gz
